In [248]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

import flask
from flask_cors import CORS

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

import os
from os import environ
import requests
import io

import datetime
from dateutil.relativedelta import relativedelta

import socket

import seaborn as sns
from itertools import groupby

In [249]:
Data=pd.read_csv('SQLsnippet_OPO.csv')
Data.columns=[x.replace(' ','') for x in Data.columns]

Data['min_stay']=Data[['is_one_way','min_stay']].apply(lambda x: -1 if x[0]==1 else x[1],axis=1)
minstay_map=DataFrame([[x for x in range(-1,11)],['one way','0 days','1 day']+[str(x)+' days' for x in range(2,11)]],index=['min_stay','min_stay_new']).transpose()

minstay_map['min_stay']=minstay_map['min_stay'].astype('int')
minstay_map['min_stay_new']=minstay_map['min_stay_new'].astype('str')

Data=Data.merge(minstay_map,on='min_stay')
Data['min_stay']=Data['min_stay_new']
Data=Data[[x for x in Data.columns if x!='min_stay_new']]

In [250]:
Data_oneway=Data.loc[Data['min_stay']=='one way',['carrier','observation_date','origin','destination','outbound_departure_date','min_stay',
'outbound_departure_time','outbound_seats','price_outbound','price_exc']]
Data_oneway['price_exc']=Data_oneway[['price_outbound','price_exc']].apply(lambda x: x[0] if pd.isnull(x[1]) else x[1],axis=1)
Data_oneway=Data_oneway[[x for x in Data_oneway.columns if x!='price_outbound']]

Data_twoway=Data.loc[Data['min_stay']!='one way',['carrier','observation_date','origin','destination','min_stay','outbound_departure_date',
'outbound_departure_time','outbound_seats','inbound_departure_date','inbound_departure_time','inbound_seats','price_outbound','price_inbound','price_exc']]

In [251]:
Data_oneway_min=Data_oneway.groupby(['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time'])['price_exc']\
.min().reset_index().rename(columns={'price_exc': 'price_outbound_min'})
Data_twoway_min=Data_twoway.groupby(['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time','inbound_departure_time'])['price_outbound','price_inbound','price_exc']\
.min().reset_index().rename(columns={'price_outbound': 'price_outbound_min','price_inbound': 'price_inbound_min','price_exc': 'price_exc_min'})

Data_twoway_mincomb=Data_twoway.merge(Data_twoway_min[['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time','inbound_departure_time','price_exc_min','price_outbound_min',
                                                                'price_inbound_min']],
on=['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time','inbound_departure_time'])

Data_twoway_mincomb=Data_twoway_mincomb.loc[Data_twoway_mincomb['price_exc']==Data_twoway_mincomb['price_exc_min'],
                                                 [x for x in Data_twoway_mincomb.columns if '_min' not in x]]

Data_twoway_mincomb=Data_twoway_mincomb.rename(columns={'price_outbound': 'price_outbound_mincomb', 'price_inbound': 'price_inbound_mincomb'})
Data_twoway_mincomb=Data_twoway_mincomb[[x for x in Data_twoway_mincomb.columns if 'price_exc' not in x]]

Data_twoway_min=Data_twoway_min.merge(Data_twoway_mincomb[['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time','inbound_departure_time','price_outbound_mincomb','price_inbound_mincomb']],
on=['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time','inbound_departure_time'])

In [252]:
Data_oneway_min=Data_oneway_min.drop_duplicates()
Data_twoway_min=Data_twoway_min.drop_duplicates()

Data_oneway_min['price_exc_min']=Data_oneway_min['price_outbound_min']

Data_twoway_min=Data_twoway_min[[x for x in Data_twoway_min if 'comb' not in x]]

#Data=pd.concat([Data_oneway_min,Data_twoway_min])

Data_oneway_min['route']=Data_oneway_min['origin']+'-'+Data_oneway_min['destination']
Data_oneway_min=Data_oneway_min[['route']+[x for x in Data_oneway_min.columns if x not in ['route','origin','destination']]]

Data_twoway_min['route']=Data_twoway_min['origin']+'-'+Data_twoway_min['destination']
Data_twoway_min=Data_twoway_min[['route']+[x for x in Data_twoway_min.columns if x not in ['route','origin','destination']]]

Data=pd.concat([Data_oneway_min,Data_twoway_min])

C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




# Dash

In [253]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [254]:
Data['outbound_departure_date']=pd.to_datetime(Data_oneway_min['outbound_departure_date'])

Data['outbound_departure_time']=Data['outbound_departure_time']\
.apply(lambda x: x if pd.isnull(x) else datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))

Data['inbound_departure_time']=Data['inbound_departure_time']\
.apply(lambda x: x if pd.isnull(x) else datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))

intermediate=Data[['route','min_stay','outbound_departure_time','inbound_departure_time','carrier']].drop_duplicates()
checkbox_values=intermediate\
.sort_values(by=['route','min_stay','outbound_departure_time','inbound_departure_time','carrier']).transpose().values.tolist()

In [255]:
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return int((dt - epoch).total_seconds())#* 1000.0

def get_marks_from_start_end(start, end):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    result = []
    current = start
    while current <= end:
        result.append(current)
        current += relativedelta(hours=1)
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in result}

def get_marks_from_start_end_2(seq):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in seq}

In [256]:
all_marks_out=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
all_marks_in=get_marks_from_start_end(min(checkbox_values[3]),max(checkbox_values[3]))

# marks_lg=get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates())
# all_marks=dict(zip([x for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000],
# [all_marks[x] for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000]))
# all_marks.update(marks_lg)

#all_marks=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))

In [257]:
PAGE_SIZE = 20

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        html.Div([html.Div(style={'width': '10%', 'display': 'inline-block'}),
        html.Img(src=app.get_asset_url('LuxairGroup.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle', 
                       'width': '10%', 'height': '7em'}),
        html.H4(children='Price Monitoring System',style={
            'textAlign': 'center',
            'color': 'black','height': '1em', 'display': 'inline-block', 'width': '64%'}),
            html.Img(src=app.get_asset_url('Luxair.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle',
                       'width': '10%', 'height': '7em'}),
        html.Div(style={'width': '10%', 'display': 'inline-block', 'vertical-align': 'middle'})
                 ],
        style={'margin-bottom': '-4em'})
     ,
    # ROUTE DROPDOWN AND DATE SLIDER
    
        # DROPDOWN
    html.Div([html.Div(style={'width': '10%', 'margin-bottom': '1em', 'display': 'inline-block'}),
    
    html.Div([html.Div([html.Label('Route', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_route',
    options=[],
    placeholder="Select Route",                                 
    value='LUX-OPO',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '50%', 'margin-left': '20em'})
    ,
    html.Div([html.Label('Minimum Stay', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_stay',
    options=[],
    placeholder="Select duration",                                 
    value='2 days',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '50%', 'margin-left': '20em'})
    ,
    html.Div([html.Label('Outbound Departure Time', style={'textAlign': 'center', 'font-weight': 'bold'}),
    html.Div([html.Label('LG', style={'textAlign': 'center'}), 
    dcc.Dropdown(id = 'dropdown_outtime',
    options=[],
    placeholder="Select departure time",                                 
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '45%', 'margin-left': '2em', 'display': 'inline-block'})
    ,
    html.Div([html.Label('Competitors', style={'textAlign': 'center', 'margin-bottom': '0em'}),
    dcc.RangeSlider(
           id = 'slider_outtime',
           updatemode = 'mouseup', #don't let it update till mouse released
           min = unix_time_millis(min(checkbox_values[2])),
           max = unix_time_millis(max(checkbox_values[2])),
           value = [unix_time_millis(min(checkbox_values[2])),
                   unix_time_millis(max(checkbox_values[2]))],
           #TODO add markers for key dates
           #marks=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2])),
           marks=all_marks_out)],
    style={'margin-bottom': '2em', 'width': '45%', 'margin-left': '2em', 'display': 'inline-block'})],          
    style={'margin-bottom': '0.5em', 'width': '90%', 'margin-left': '0em'})
    ,
    html.Div([html.Label('Inbound Departure Time', style={'textAlign': 'center', 'font-weight': 'bold'}),
    html.Div([html.Label('LG', style={'textAlign': 'center'}), 
    dcc.Dropdown(id = 'dropdown_intime',
    options=[],
    placeholder="Select departure time",                                 
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '45%', 'margin-left': '4em', 'display': 'inline-block'})
    ,
    html.Div([html.Label('Competitors', style={'textAlign': 'center', 'margin-bottom': '0em'}),
    dcc.RangeSlider(
           id = 'slider_intime',
           updatemode = 'mouseup', #don't let it update till mouse released
           min = unix_time_millis(min(checkbox_values[2])),
           max = unix_time_millis(max(checkbox_values[2])),
           value = [unix_time_millis(min(checkbox_values[2])),
                   unix_time_millis(max(checkbox_values[2]))],
           #TODO add markers for key dates
           #marks=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2])),
           marks=all_marks_in)],
    style={'margin-bottom': '2em', 'width': '45%', 'margin-left': '2em', 'display': 'inline-block'})],          
    style={'margin-bottom': '0.5em', 'width': '90%', 'margin-left': '2em'})    
             ],
    style={'width': '75%', 'margin-bottom': '1em', 'margin-top': '0','display': 'inline-block', 'background-color': 'lightgrey'}),
    
    html.Div(style={'width': '10%', 'margin-bottom': '0.5em', 'margin-top': '-3em', 'display': 'inline-block'})
             ]),
    # GRAPH CONTAINER                   
    html.Div([html.Div([dcc.Graph(
                    id='graph-outbound')],
    style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'margin-top': '0.5em'}),
    html.Div([dcc.Graph(
                    id='graph-inbound')],
    style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'margin-top': '0.5em'})
             ])
        ])

In [258]:
@app.callback(
    Output("dropdown_route", "options"),
    [Input("dropdown_stay", "value")]
    )
def set_checklist_route(values_stay):
    
    subset=set([checkbox_values[0][i] for i in range(len(checkbox_values[0]))\
    if checkbox_values[1][i]==values_stay])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [259]:
@app.callback(
    Output("dropdown_stay", "options"),
    [Input("dropdown_route", "value")]
    )
def set_checklist_stay(values_route):
    
    subset=set([checkbox_values[1][i] for i in range(len(checkbox_values[1]))\
    if checkbox_values[0][i]==values_route])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [260]:
@app.callback(
    Output("dropdown_outtime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value")]
    )
def set_checklist_outtime(values_route,values_stay):
    
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i]==values_route\
    and checkbox_values[1][i]==values_stay\
    and checkbox_values[4][i]=='LG'])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [261]:
@app.callback(
    Output("dropdown_intime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value")
    #,
    #Input("dropdown_outtime", "value")
    ]
    )
def set_checklist_intime(values_route,values_stay):
                         #,values_outtime):
    
    subset=set([checkbox_values[3][i] for i in range(len(checkbox_values[3]))\
    if checkbox_values[0][i]==values_route\
    and checkbox_values[1][i]==values_stay\
    #and checkbox_values[2][i]==values_outtime\
    and checkbox_values[4][i]=='LG'])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [262]:
@app.callback(
    [Output("slider_outtime", "min"),
    Output("slider_outtime", "max"),
    Output("slider_outtime", "marks")],
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value")]
    )
def set_checklist_depdate(values_route,values_stay):
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i]==values_route\
    and checkbox_values[1][i]==values_stay\
    #and checkbox_values[2][i]==values_outtime\
    #and checkbox_values[4][i]=='LG'
    ])
    minimum=min([unix_time_millis(x) for x in subset])
    maximum=max([unix_time_millis(x) for x in subset])
    value=[minimum,maximum]
    marks=get_marks_from_start_end([x for x in subset if unix_time_millis(x)==minimum][0],
                                   [x for x in subset if unix_time_millis(x)==maximum][0])
    return minimum, maximum, marks

In [263]:
colors_carrier=dict(zip(Data['carrier'].unique(),['yellow','blue','red','orange']))
line_inout=dict(zip(['price_outbound_min','price_outbound_min'],['dash','dot']))

In [ ]:
@app.callback(Output('graph-outbound', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input('dropdown_route', 'value'),
    Input('dropdown_stay', 'value'),
    Input('dropdown_outtime', 'value')]
    )
def update_plot(value_route, value_stay, value_outtime):
    
    dff_plot = Data.loc[(Data['route']=='value_route')\
                   &Data['value_stay']==''.apply(lambda x: unix_time_millis(x)>=values_deptime[0] and unix_time_millis(x)<=values_deptime[1])\
                   &Data['min_stay'].apply(lambda x: x in values_stay),:].sort_values(by=['outbound_departure_date'])
    carriers=dff_plot['carrier'].unique()
    
    traces = []
        
    for i in ['price_outbound_min','price_inbound_min']:
        for j in carriers:
            traces.append(go.Scatter(
               x=dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_date'].tolist(),
               y=dff_plot.loc[dff_plot['carrier']==j,i].tolist(),
               mode = 'lines',
               name = j+'/'+i,
               line = dict(
               dash = line_inout[i],
               color =colors_carrier[j],
               width = 2
                 )
            ))
   
        return {'data': traces, 'layout': 
        go.Layout(title=go.layout.Title(
        text='tst',
        xref='paper',
        x=0.5),
        #xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        #yaxis={'title': 'Bid Price', 'range': [0,dff_plot['price_exc_min'].max()+5]},
        height=600)}
    else:   
        return {'data': []}         

In [ ]:
@app.callback(Output('graph-inbound', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input('dropdown_route', 'value'),
    Input('deptime_RangeSlider', 'value'),
    Input('dropdown_stay', 'value')]
    )
def update_plot(values_route, values_deptime, values_stay):
    
    dff_plot = Data.loc[Data['route'].apply(lambda x: x in values_route)\
                   &Data['outbound_departure_time'].apply(lambda x: unix_time_millis(x)>=values_deptime[0] and unix_time_millis(x)<=values_deptime[1])\
                   &Data['min_stay'].apply(lambda x: x in values_stay),:].sort_values(by=['outbound_departure_date'])
    carriers=dff_plot['carrier'].unique()
    
    traces = []
        
    for i in ['price_outbound_min','price_inbound_min']:
        for j in carriers:
            traces.append(go.Scatter(
               x=dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_date'].tolist(),
               y=dff_plot.loc[dff_plot['carrier']==j,i].tolist(),
               mode = 'lines',
               name = j+'/'+i,
               line = dict(
               dash = line_inout[i],
               color =colors_carrier[j],
               width = 2
                 )
            ))
   
        return {'data': traces, 'layout': 
        go.Layout(title=go.layout.Title(
        text='tst',
        xref='paper',
        x=0.5),
        #xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        #yaxis={'title': 'Bid Price', 'range': [0,dff_plot['price_exc_min'].max()+5]},
        height=600)}
    else:   
        return {'data': []}         

In [ ]:
if __name__ == '__main__':
    #app.run_server(debug=False,host='10.0.0.8',port=port[analyst])
    app.run_server(debug=False,host='localhost',port=4000)